### Exploration for understanding Data

This will not run here since the dataclasses, and functions are not here. They are in the main file.

### Imports


In [ ]:
import json

### Exploring characters & their moves throughout the game

In [ ]:
# needed to put entry info into JSON 
class EntryInfoEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, EntryInfo):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

In [ ]:
# processed_story.characters is how to access the characters
print(f"Here are all character ids: {processed_story.characters.keys()}\n") # all characters are the keys 

for char_info in processed_story.characters.values():  # value is the characterInfo object
    print(f"The character is: {char_info.name}")
    print(f"Associated things stored with each character: {char_info.__annotations__.keys()}\n\n") # these are the associated things stored with each character
    print(f"Here are the entry ids/moves by this character (IN ORDER) throughout this game: \n  {char_info.entry_ids}\n")

    info_all_entries_by_current_character = []

    # getting data for all entries done by this character
    for entry_id in char_info.entry_ids:
        info_all_entries_by_current_character.append(processed_story.entries[entry_id])
    
    # writing all data to JSON file (labelled with this character's name)
    with open(f"{char_info.name}_entry_info.json", "w") as file:
        json.dump(info_all_entries_by_current_character, file, cls=EntryInfoEncoder, indent=4)
    
    print(f"Details of all these entries written to file \"{char_info.name}_entry_info.json\"")


### Exploring entries throughout the game (in sequence)

In [ ]:
# processed_story.entries is how to access the entries
# processed_story.establishment_entries is how to access the scene start entries

print(f"Here are all entries in this game, by everyone (narrator or other chars): {processed_story.entries.keys()}\n") 

# Keeping track of last known location since some starting scenes do not have place cards, so we continue the previous scene's location
# It is initialized to the first place in Scene 1 of Game, it is always present
first_scene_loc = list(processed_story.establishment_entries.values())[0].imp_cards['place_card'][0]
last_known_location = {"card_id": first_scene_loc["card_id"], "name": first_scene_loc["name"], "description": first_scene_loc["description"]}  

for entry_info in processed_story.entries.values():  # value is the entryInfo object

    print(f"Associated things stored with each entry: {entry_info.__annotations__.keys()}\n\n")

    # when establishment entry
    if entry_info.format == "establishment":
        print("This is a scene start entry, description of this is our initial paragraph for this scene\n")
        print("This para will repeat with each entry, since there are multiple entries in a scene")
        print(f"Initial para: \t {entry_info.text}")
    
        # setting place for each scene
        scene_location = entry_info.imp_cards["place_card"][0]
        if scene_location is not None:
            last_known_location = {"card_id": scene_location["card_id"], "name": scene_location["name"], "description": scene_location["description"]}
        else:
            print(f"entry_id: {entry_id} is establishment but has no place card..")

        print(f"The place associated with this scene will also repeat across parts of scene: {last_known_location["name"]}")
    elif entry_info.format == "move":

        print(f"Each \"Move\" format entry's content will be the Current paragraph. Concatentated with initial para, it forms our total context\n")
        total_context = processed_story.establishment_entries[entry_info.establishment_id].text + entry_info.text
        print(f"Here is the total context: {total_context}")
        character = processed_story.characters[entry_info.character_id]
        # character.character_id, character.name, character.description for JSON later
        print(f"The character in this part of the scene is: \t {character.name}")

        for event in entry_info.imp_cards["cards_played_on_challenge"]:
            print(f"Here is an event this character does in this context \t {event}")
        
    # scene conclusion and addition entries by narrator are ignored (since there is no event/character options to them for BERT)
    else:
        continue